# AI Traffic System Recognition System

Description.

## Necessary Packages

In [12]:
!pip install ipywidgets tensorflow opencv-python-headless Pillow pandas numpy matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.3 MB/s eta 0:00:0000:0100:01


In [14]:
# Importing the packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io

## Data Loading and Preprocessing

I found the GTSRB on Kaggle. Using Kaggle API, I download the data into the notebook.

In [1]:
!pip install kaggle
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
100%|████████████████████████████████████████| 612M/612M [00:20<00:00, 37.0MB/s]
100%|████████████████████████████████████████| 612M/612M [00:20<00:00, 30.9MB/s]


Next, the data is extracted:

In [36]:
import zipfile
import os

zip_path = 'gtsrb-german-traffic-sign.zip'
extract_to = 'data'

# Create the directory
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction Done")

Extraction Done


Preprocessing:

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2

# Define image dimensions (e.g., 32x32)
img_width, img_height = 32, 32

def load_images_and_labels_one(base_path):
    images = []
    labels = []
    num_classes = 43  # There are 43 classes of traffic signs in the GTSRB dataset

    for folder in range(num_classes):
        path = os.path.join(base_path, str(folder))
        if not os.path.exists(path):
            continue
        for file in os.listdir(path):
            image_path = os.path.join(path, file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (img_width, img_height))
            images.append(image)
            labels.append(folder)
    
    # Convert lists to numpy arrays and normalize image vectors
    images = np.array(images)
    images = images.astype('float32') / 255.0
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    
    return images, labels

def load_test_images(base_path):
    images = []
    files = [os.path.join(base_path, file) for file in os.listdir(base_path) if file.endswith('.png')]
    files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))  # Ensure files are sorted numerically
    
    for image_path in files:
        image = cv2.imread(image_path)
        image = cv2.resize(image, (img_width, img_height))
        images.append(image)
    
    # Convert list to numpy array and normalize image vectors
    images = np.array(images)
    images = images.astype('float32') / 255.0
    
    return images

# Extracting the dataset to 'data/train' and 'data/test'
train_images, train_labels = load_images_and_labels('data/train')
test_images = load_test_images('data/test')

print(f"Train images shape: {train_images.shape}")
print(f"Test images shape: {test_images.shape}")
print("Data Loaded and Processed")

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


Model Building

In [32]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
])

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 43)             │        22,059 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 377,963 (1.44 MB)

 Trainable params: 377,963 (1.44 MB)

 Non-trainable params: 0 (0.00 B)

## Training

In [33]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator, verbose=1)

NameError: name 'test_generator' is not defined

## Testing